In [1]:
# =============================================
# Phase 2 - Applied Quant & AI
# Shared Data Gathering Notebook
# =============================================

# Author: Rein | Pixel Sky Solutions
# Course: Python–Finance Master Plan
# ---------------------------------------------
# In this notebook, we collect and standardize
# financial datasets from multiple sources:
#   • Market data (Yahoo Finance via yahooquery)
#   • Macroeconomic indicators (World Bank API)
#   • Company filings (SEC EDGAR API)
#
# Output:
#   Cleaned CSV files ready for ML pipeline use
#   -> market_data.csv
#   -> macro_data.csv
#   -> sec_filings.csv
# ---------------------------------------------

In [2]:
# =====================================================
# 1️⃣  SETUP & IMPORTS
# =====================================================

In [5]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from yahooquery import Ticker
import wbdata
import time
import os

In [6]:
# Always display all columns
pd.set_option('display.max_columns', None)

In [7]:
# Define data folder
os.makedirs("data_phase2", exist_ok=True)

In [8]:
# =====================================================
# 2️⃣  YAHOO FINANCE DATA
# =====================================================
# We'll pull price & fundamentals for a few tickers.
# yahooquery makes it easy to fetch bulk market data.

In [9]:
tickers = ["AAPL", "MSFT", "TSLA", "SPY"]

In [10]:
print("Fetching daily OHLCV data...")
# Ticker.history(period='2y') gives 2 years by default.
data_yahoo = (
    Ticker(tickers)
    .history(period="2y", interval="1d", adj_ohlc=True)
    .reset_index()
)

Fetching daily OHLCV data...


In [11]:
# Clean and rename columns
data_yahoo.rename(columns={'symbol': 'Ticker'}, inplace=True)

In [13]:
# Keep essential columns only
market_data = data_yahoo[['date', 'Ticker', 'open', 'high', 'low', 'close', 'volume']]
market_data.to_csv("data_phase2/market_data.csv", index=False)

print(f"✅ Market data saved: {market_data.shape} rows")

✅ Market data saved: (2008, 7) rows


In [14]:
# =====================================================
# 3️⃣  WORLD BANK MACROECONOMIC DATA
# =====================================================
# We'll fetch GDP growth, inflation, and interest rate
# for the US as macroeconomic context variables.

In [21]:
print("Fetching macroeconomic indicators from World Bank...")

# Define parameters
country = "US"
indicators = {
    "NY.GDP.MKTP.KD.ZG": "GDP_Growth",
    "FP.CPI.TOTL.ZG": "Inflation",
    "FR.INR.LEND": "Interest_Rate",
}

Fetching macroeconomic indicators from World Bank...


In [22]:
start_year = dt.date.today().year - 10
end_year = dt.date.today().year

all_frames = []

In [23]:
# Loop through indicators manually via REST API
for code, name in indicators.items():
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{code}?format=json&per_page=1000"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"❌ Failed to fetch {name}: {response.status_code}")
        continue

    json_data = response.json()
    if not json_data or len(json_data) < 2:
        print(f"⚠️ No data returned for {name}")
        continue

    data = pd.DataFrame(json_data[1])[["date", "value"]]
    data["Indicator"] = name
    data = data.rename(columns={"date": "Year", "value": name})
    data["Year"] = data["Year"].astype(int)
    data = data[(data["Year"] >= start_year) & (data["Year"] <= end_year)]
    all_frames.append(data[["Year", name]])

In [24]:
# Merge all indicators on Year
if all_frames:
    macro_data = all_frames[0]
    for df in all_frames[1:]:
        macro_data = pd.merge(macro_data, df, on="Year", how="outer")

    macro_data.sort_values("Year", inplace=True)
    macro_data.reset_index(drop=True, inplace=True)
    macro_data.to_csv("data_phase2/macro_data.csv", index=False)
    print(f"✅ Macro data saved: {macro_data.shape} rows")
else:
    print("⚠️ No macroeconomic data could be fetched.")

✅ Macro data saved: (10, 4) rows


In [25]:
# =====================================================
# 4️⃣  SEC FILINGS DATA
# =====================================================
# We'll use the official SEC EDGAR API to pull basic
# filing metadata (10-K, 10-Q) for selected tickers.
# Note: SEC has rate limits — sleep between requests.

In [26]:
print("Fetching SEC filings metadata...")

# Function to fetch SEC filings for a ticker
def get_sec_filings(ticker, form_type="10-K", limit=10):
    """
    Fetch latest filings metadata from SEC EDGAR API.
    Parameters:
        ticker (str): Company ticker symbol.
        form_type (str): Form type ('10-K', '10-Q', etc.)
        limit (int): Number of filings to retrieve.
    """
    url = f"https://data.sec.gov/submissions/CIK0000320193.json"
    # In practice you’d map ticker→CIK (can use sec_api.com or manual file)
    # We'll use Apple (AAPL, CIK0000320193) for demonstration.

    headers = {
        "User-Agent": "PixelSkySolutions (rein@pixelsky.solutions)"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Failed {ticker}: {response.status_code}")
        return None

    data = response.json()
    filings = pd.DataFrame(data.get("filings", {}).get("recent", {}))
    filings = filings.head(limit)
    filings["ticker"] = ticker
    return filings[["ticker", "form", "filingDate", "reportDate", "accessionNumber"]]

Fetching SEC filings metadata...


In [27]:
# Example for Apple (as EDGAR demo)
sec_data = get_sec_filings("AAPL")
if sec_data is not None:
    sec_data.to_csv("data_phase2/sec_filings.csv", index=False)
    print(f"✅ SEC filings saved: {sec_data.shape} rows")
else:
    print("⚠️ No SEC data fetched (demo mode).")

✅ SEC filings saved: (10, 5) rows


In [28]:
# =====================================================
# 5️⃣  DATA CHECKPOINT SUMMARY
# =====================================================

In [29]:
print("\n--- Data Summary ---")
print("Market data sample:")
print(market_data.head(), "\n")
print("Macro data sample:")
print(macro_data.head(), "\n")
print("SEC data sample:")
print(sec_data.head() if sec_data is not None else "None")

print("\nAll raw data stored in ./data_phase2/")


--- Data Summary ---
Market data sample:
         date Ticker        open        high         low       close    volume
0  2023-10-11   AAPL  176.471483  178.105487  175.877312  178.055969  47551100
1  2023-10-12   AAPL  178.323347  180.571317  177.303324  178.957138  56743100
2  2023-10-13   AAPL  179.660252  180.165300  176.412069  177.115189  51427100
3  2023-10-16   AAPL  175.035543  177.342944  174.797865  176.986435  52517000
4  2023-10-17   AAPL  174.936521  176.689357  173.104475  175.431671  57549400 

Macro data sample:
   Year  GDP_Growth  Inflation  Interest_Rate
0  2015    2.945550   0.118627       3.260000
1  2016    1.819451   1.261583       3.511667
2  2017    2.457622   2.130110       4.096667
3  2018    2.966505   2.442583       4.904167
4  2019    2.583825   1.812210       5.282500 

SEC data sample:
  ticker form  filingDate  reportDate       accessionNumber
0   AAPL    4  2025-10-03  2025-10-01  0001214156-25-000011
1   AAPL    4  2025-10-03  2025-10-01  000176709